<a href="https://colab.research.google.com/github/Darkblazer21/vehicle_classifier/blob/dev/vehicle_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building A Vehicle Classifier Using Fastai

### By Brems Mbodj

Let's start by installing Fastai In Our Colab Notebook

In [2]:
!pip install -Uqq fastbook

# Importing the fastai vision library

This allows to get all relevant functions and classes for our image classification tasks

In [3]:
from fastai.vision.all import *

# Gathering the data

Let's now gather some data. Our different image classes are:
- car
- motorcycle
- truck
- bus
- bike
- train
- scooter

To download images we'll have to install `jmd_imagescraper` library which will let's gather images using the `duck duck go` search api

In [17]:
!pip install -q jmd_imagescraper

Let's now import all the relevant libraries in order to successfully download images using the `jmd_imagescraper`

In [19]:
from pathlib import Path
from jmd_imagescraper.core import *

In [20]:
root = Path().cwd()/"images" # We instantiate the Path object for our images